#### 다나와 크롤링

##### 다나와 무선청소기 크롤링

- 무선청소기 URL : https://search.danawa.com/dsearch.php?query=%EB%AC%B4%EC%84%A0%EC%B2%AD%EC%86%8C%EA%B8%B0&page=1&limit=120

##### 셀레니움 등  크롤링 모듈 임포트

In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import pandas as pd
from tqdm.autonotebook import tqdm

In [2]:
# tqdm progressbar 워닝때문에 설치
!pip install ipywidgets

                                              0.0/138.3 kB ? eta -:--:--
     -------------------------------------- 138.3/138.3 kB 4.1 MB/s eta 0:00:00
                                              0.0/2.3 MB ? eta -:--:--
     -----------                              0.7/2.3 MB 13.9 MB/s eta 0:00:01
     -------------------------                1.5/2.3 MB 15.7 MB/s eta 0:00:01
     ---------------------------------------  2.3/2.3 MB 18.3 MB/s eta 0:00:01
     ---------------------------------------- 2.3/2.3 MB 16.2 MB/s eta 0:00:00
                                              0.0/215.0 kB ? eta -:--:--
     ------------------------------------- 215.0/215.0 kB 13.6 MB/s eta 0:00:00


In [5]:
# 크롬드라이버로 크롬 브라우저 실행
driver = webdriver.Chrome()
url = 'https://search.danawa.com/dsearch.php?query=%EB%AC%B4%EC%84%A0%EC%B2%AD%EC%86%8C%EA%B8%B0&page=1&limit=120'
driver.get(url)

In [9]:
# bs 사용 html 가져오기
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [12]:
# 제품리스트만
prod_items = soup.select('li.prod_item')
len(prod_items)

123

In [28]:
# 첫번째 제품으로 값가져오기 체크
# 1. 제품명 가져오기
# strip() - 공백, \n 다 없애줌 / 중간에 애드스마트 광고 부분 예외처리 제거대상
title = prod_items[0].select('p.prod_name')[0].text.strip()
title

'삼성전자 비스포크 제트 VS20B956D5'

In [30]:
# 2. 제품 스펙 가져오기
spec_list = prod_items[0].select('div.spec_list')[0].text.strip().replace('\t','') 
spec_list

'핸디스틱청소기 / 무선 / 흡입+물걸레겸용 / 흡입력: 220W / [구성] 거치대: 일체형청정스테이션 / 바닥 / 물걸레 / 솔형 / 틈새 / 연장툴 / [배터리] 분리형 (1개) / 충전시간: 3시간30분 / 사용시간: 1시간 / [성능] 디지털인버터모터 / 5단계여과 / [기능] IoT기능 / 디스플레이표시창 / 자동물공급 / LED라이트 / 배터리잔량표시 / 자동먼지비움 / [부가] 먼지통용량: 0.5L / 색상: 산토리니베이지 / 2022년형 / 물걸레 / 크기(가로x세로x깊이): 250x930x202mm'

In [36]:
# 3. 가격 정보 가져오기
price = prod_items[0].select('li.rank_one > p.price_sect > a > strong')[0].text.strip().replace(',','')
price

'555480'

In [47]:
# 애드스마트 광고영역을 판별
'ad_section' in prod_items[5].select('div')[0]['class']

True

##### 반복문으로 검색결과 상품정보 리스트 추출

In [61]:
prod_data = []
# count = 0

for prod_item in prod_items:
    # if count >= 7 : break

    if 'ad_section' in prod_item.select('div')[0]['class']:
        continue # 애드스마트 영역이면 패스
    try:
        title = spec_list = prod_item.select('p.prod_name > a')[0].text.strip() # 1. 제품명
        spec_list = prod_item.select('div.spec_list')[0].text.strip().replace('\t','') # 2. 제품스펙
        price = prod_item.select('li.rank_one > p.price_sect > a > strong')[0].text.strip().replace(',','') # 제품가격
        prod_data.append([title, spec_list, price])
    except:
        pass

    # count += 1


In [62]:
len(prod_data)

120

In [64]:
pd.DataFrame(prod_data).head()

,0,1,2
0,삼성전자 비스포크 제트 VS20B956D5,핸디스틱청소기 / 무선 / 흡입+물걸레겸용 / 흡입력: 220W / [구성] 거치대...,555480
1,LG전자 오브제컬렉션 코드제로 A9S AX9984,핸디스틱청소기 / 무선 / 흡입+물걸레겸용 / 흡입력: 250W / [구성] 거치대...,1136630
2,LG전자 오브제컬렉션 코드제로 AS9202WD,핸디스틱청소기 / 무선 / 흡입전용 / 흡입력: 210W / [구성] 거치대: 거치...,530620
3,샤오미 드리미 V11,핸디스틱청소기 / 무선 / 흡입전용 / 흡입력: 150AW / 흡입력: 25000P...,179000
4,삼성전자 비스포크 제트 VS20A956A3,핸디스틱청소기 / 무선 / 흡입전용 / 흡입력: 210W / [구성] 거치대: 일체...,482660


In [71]:
# 함수만들기
def get_prod_items(prod_items):
    prod_data = []

    for prod_item in prod_items:

        if 'ad_section' in prod_item.select('div')[0]['class']:
            continue # 애드스마트 영역이면 패스
        try:
            title = spec_list = prod_item.select('p.prod_name > a')[0].text.strip() # 1. 제품명
            spec_list = prod_item.select('div.spec_list')[0].text.strip().replace('\t','') # 2. 제품스펙
            price = prod_item.select('li.rank_one > p.price_sect > a > strong')[0].text.strip().replace(',','') # 제품가격
            prod_data.append([title, spec_list, price])
        except:
            pass

    return prod_data

In [73]:
# 함수 테스트
prod_items1 = soup.select('li.prod_item')
prod_data1 = get_prod_items(prod_items1)

In [75]:
pd.DataFrame(prod_data1)

,0,1,2
0,삼성전자 비스포크 제트 VS20B956D5,핸디스틱청소기 / 무선 / 흡입+물걸레겸용 / 흡입력: 220W / [구성] 거치대...,555480
1,LG전자 오브제컬렉션 코드제로 A9S AX9984,핸디스틱청소기 / 무선 / 흡입+물걸레겸용 / 흡입력: 250W / [구성] 거치대...,1136630
2,LG전자 오브제컬렉션 코드제로 AS9202WD,핸디스틱청소기 / 무선 / 흡입전용 / 흡입력: 210W / [구성] 거치대: 거치...,530620
3,샤오미 드리미 V11,핸디스틱청소기 / 무선 / 흡입전용 / 흡입력: 150AW / 흡입력: 25000P...,179000
4,삼성전자 비스포크 제트 VS20A956A3,핸디스틱청소기 / 무선 / 흡입전용 / 흡입력: 210W / [구성] 거치대: 일체...,482660
...,...,...,...
115,일렉트로룩스 ZB6218STM,핸디청소기 / 무선 / 흡입전용 / [구성] 거치대: 충전+거치 / [배터리] 내장...,147150
116,한빛시스템 HV-5099,핸디스틱청소기 / 무선 / 흡입전용 / [구성] 바닥 / 솔형 / 틈새 / [배터리...,75090
117,캐치웰 CX PRO,핸디스틱청소기 / 무선 / 흡입전용 / [구성] 바닥 / 침구 / 솔형 / 틈새 /...,329000
118,LG전자 코드제로 A9S AS9271,핸디스틱청소기 / 무선 / 흡입+물걸레겸용 / 흡입력: 210W / [구성] 거치대...,677310


##### 페이지 변경하면서 크롤링

In [82]:
# 페이지 URL 생성함수
def get_search_page_url(keyword, page):
    return f'https://search.danawa.com/dsearch.php?query={keyword}&page={page}&limit=120&sort=saveDESC'

In [85]:
# 함수 테스트
keyword = '무선청소기'
page = 150
url = get_search_page_url(keyword, page)
driver.get(url)

##### 여러페이지 상품 데이터 수집

In [98]:
driver = webdriver.Chrome()
# 암묵적으로 웹자원 로드를 위해서 3초 대기
# driver.implicitly_wait(3)

In [99]:
from time import sleep

total_page = 20
prod_total_data = []

for page in tqdm(range(1,total_page + 1)):
    url = get_search_page_url(keyword='무선청소기', page=page)
    driver.get(url)
    sleep(7.0) # 5초로 대기했을때 못가져오는 데이터 발생

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    prod_items = soup.select('li.prod_item')
    prod_item_list = get_prod_items(prod_items)

    prod_total_data += prod_item_list

100%|██████████| 20/20 [04:28<00:00, 13.45s/it]


In [100]:
pd.DataFrame(prod_total_data)

,0,1,2
0,삼성전자 비스포크 제트 VS20B956D5,핸디스틱청소기 / 무선 / 흡입+물걸레겸용 / 흡입력: 220W / [구성] 거치대...,555480
1,LG전자 오브제컬렉션 코드제로 A9S AX9984,핸디스틱청소기 / 무선 / 흡입+물걸레겸용 / 흡입력: 250W / [구성] 거치대...,1136630
2,LG전자 오브제컬렉션 코드제로 AS9202WD,핸디스틱청소기 / 무선 / 흡입전용 / 흡입력: 210W / [구성] 거치대: 거치...,530620
3,샤오미 드리미 V11,핸디스틱청소기 / 무선 / 흡입전용 / 흡입력: 150AW / 흡입력: 25000P...,179000
4,삼성전자 비스포크 제트 VS20A956A3,핸디스틱청소기 / 무선 / 흡입전용 / 흡입력: 210W / [구성] 거치대: 일체...,482660
...,...,...,...
2220,라미 LED차량용 무선 청소기,"차량용청소기 / 무선 / [흡입력] ~6,000Pa / [필터] 헤파필터 / 스테인...",69000
2221,정인일렉텍 큐브온 핸디형 무선 진공 청소기,"차량용청소기 / 무선 / [흡입력] 6,000Pa / [소비전력] 70W / [필터...",53990
2222,바이거 DEX-WC230,물걸레청소기 / 회전식 / 물걸레전용 / 무선 / [배터리] 사용시간: 1시간20분...,146700
2223,앤커 무선 핸디형 청소기 유피 홈백 H11,"차량용청소기 / 무선 / [흡입력] 5,500Pa / [소비전력] 70W / [필터...",86900


In [97]:
# 수집데이터 저장
df = pd.DataFrame(prod_total_data)
df.columns = ['상품명', '스펙목록', '가격']
df.to_excel('./1_다나와_무선청소기크롤링결과.xlsx', index=False)